# Imports

In [1]:
import torch

# Debug WBCE-by-disease

In [2]:
from torch import nn
from torch.nn.functional import cross_entropy, softmax, binary_cross_entropy_with_logits

In [ ]:
%run ./wbce.py

In [ ]:
loss = WeigthedBCEByDiseaseLoss()

In [ ]:
bs = 5
out = torch.randn(bs, 4)
target = (torch.rand(bs, 4) > 0.75).long()
out, torch.sigmoid(out), target

In [ ]:
loss(out, target)

## BCE with weights

In [ ]:
bs = 3
out = torch.randn(bs, 4)
target = (torch.rand(bs, 4) > 0.75).long()
out, torch.sigmoid(out), target

In [ ]:
loss = nn.BCEWithLogitsLoss(reduction='none')
loss(out, target.float())

In [ ]:
loss.to('cuda')

In [ ]:
loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1, 2, 3, 4]), reduction='none')
loss(out, target.float())

# Debug Focal loss

In [ ]:
%run focal.py

In [ ]:
alpha = 0.75
gamma = 2

## Multilabel case

In [ ]:
outputs = torch.tensor([[1, 1, 0, 0, 0, 1],
                        [0, 0, 1, 0, 0, 1],
                       ]).float()
targets = torch.tensor([[1, 1, 1, 0, 1, 0],
                        [0, 1, 0, 0, 1, 0],
                       ]).float()
outputs

In [ ]:
focal = FocalLoss(alpha=0.87, reduction='none')
focal(outputs, targets)

## Multiclass case

In [ ]:
outputs = torch.tensor([[0, 1, 0],
                        [-3, 1, 2],
                        [1, 11.1, 0.5],
                       ]).float()
targets = torch.tensor([1, 2, 0]).long()

In [ ]:
focal = FocalLoss(alpha=0.9, gamma=2, multilabel=False, reduction='none')
focal(outputs, targets)

# Debug out-of-target

In [37]:
import numpy as np

In [3]:
%run ./out_of_target.py

In [4]:
l = OutOfTargetSumLoss()
l

OutOfTargetSumLoss()

In [30]:
image1 = [[0, 0, 0.0], [0, 1, 0]]
target1 = [[0, 1, 0], [1, 0, 1]]

image2 = [[0.2, 0.8, 0.0], [0.1, 0.3, 0]]
target2 = [[0, 1, 1], [1, 1, 1]]

In [31]:
def calc_for_one(i, t):
    out = torch.tensor(i).unsqueeze(0).unsqueeze(0)
    target = torch.tensor(t).unsqueeze(0).unsqueeze(0)

    return l(out, target)

In [49]:
results = [
    calc_for_one(image1, target1),
    calc_for_one(image2, target2),
    calc_for_one(image2, target2),
]
results, np.mean(results)

([tensor(6.1402), tensor(0.2231), tensor(0.2231)], 2.1955047)

In [52]:
new_dim = 0

out = torch.stack((
    torch.tensor(image1),
    torch.tensor(image2),
    torch.tensor(image2),
), dim=0).unsqueeze(new_dim)
target = torch.stack((
    torch.tensor(target1),
    torch.tensor(target2),
    torch.tensor(target2),
), dim=0).unsqueeze(new_dim)
out.size(), target.size()

(torch.Size([1, 3, 2, 3]), torch.Size([1, 3, 2, 3]))

In [53]:
l(out, target)

tensor(2.1955)

# WBCE with images

In [124]:
%run ./wbce.py

In [125]:
l = WeigthedBCELoss(sigmoid=False)

In [160]:
bs = 5
n_diseases = 4
h = w = 100
target = (torch.rand(bs, n_diseases, h, w) > 0.9).long()
# out = torch.randn(bs, n_diseases, h, w)
# out = torch.ones(bs, n_diseases, h, w)
# out = torch.sigmoid(out)
out = (target.float() + torch.randn(*target.size()) * 0.5).clamp(0, 1)
out.size(), target.size(), f'{target.sum().item():,}'

(torch.Size([5, 4, 100, 100]), torch.Size([5, 4, 100, 100]), '19,912')

In [161]:
l(out, target)

10.04419445560466 1.1105681666740705


tensor(1.2946)